***BUILDING AN ARTIFICIAL NEURAL NETWORK***

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
#set random seeds for reproducibility
torch.manual_seed(42)

In [ ]:
df=pd.read_csv('/content/Churn_Modelling (1).csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
#train test split
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train

array([[15601116, "P'an", 686, ..., 1, 179093.26, 0],
       [15766374, 'Leak', 632, ..., 1, 195978.86, 0],
       [15716994, 'Green', 559, ..., 0, 85891.02, 1],
       ...,
       [15573851, 'Macrossan', 735, ..., 0, 92220.12, 1],
       [15807663, 'McGregor', 667, ..., 0, 97508.04, 1],
       [15706268, 'Smith', 697, ..., 1, 53581.14, 0]], dtype=object)

In [ ]:
X_test

array([[15687492, 'Anderson', 596, ..., 0, 41788.37, 0],
       [15736963, 'Herring', 623, ..., 1, 146379.3, 0],
       [15721730, 'Amechi', 601, ..., 0, 58561.31, 0],
       ...,
       [15746674, 'Miller', 730, ..., 0, 33373.26, 1],
       [15689081, 'Wu', 692, ..., 0, 76755.99, 1],
       [15651627, 'White', 628, ..., 1, 107674.3, 1]], dtype=object)

In [ ]:
y_train

array([9255, 1562, 1671, ..., 5391,  861, 7271])

In [ ]:
y_test

array([6253, 4685, 1732, ..., 7854, 1096, 6930])

In [ ]:
#create CustomDataset Class
class CustomDataset(Dataset):
  def __init__(self,X,y):
    self.X=torch.tensor(X,dtype=torch.float32)
    self.y=torch.tensor(y,dtype=torch.long)

  def __len__(self):
    return len(self.X)

  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [ ]:
#create train_dataset object
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

# Original X columns (derived from df.iloc[:,1:]):
# 0: CustomerId, 1: Surname, 2: CreditScore, 3: Geography, 4: Gender,
# 5: Age, 6: Tenure, 7: Balance, 8: NumOfProducts, 9: HasCrCard,
# 10: IsActiveMember, 11: EstimatedSalary, 12: Exited (this is the target)

# Create copies to avoid modifying original X_train, X_test if they are used elsewhere
X_train_copy = np.copy(X_train)
X_test_copy = np.copy(X_test)

# Drop CustomerId (original index 0) and Surname (original index 1)
X_train_temp = np.delete(X_train_copy, [0, 1], axis=1)
X_test_temp = np.delete(X_test_copy, [0, 1], axis=1)

# After dropping, the new indices in X_train_temp and X_test_temp are:
# 0: CreditScore, 1: Geography (original 3), 2: Gender (original 4),
# 3: Age (original 5), ..., 10: Exited (original 12)

# Identify categorical features for encoding in the *temp* arrays
categorical_features_indices_in_temp = [1, 2] # Geography (new index 1), Gender (new index 2)

# Apply OneHotEncoder to categorical features
# sparse_output=False ensures a dense numpy array output
ct = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_indices_in_temp)
    ],
    remainder='passthrough' # Keep other columns as they are
)

# Fit and transform on X_train_temp, then transform X_test_temp
X_train_processed = ct.fit_transform(X_train_temp)
X_test_processed = ct.transform(X_test_temp)

# Ensure all features are float32 for PyTorch
X_train_processed = X_train_processed.astype(np.float32)
X_test_processed = X_test_processed.astype(np.float32)

# Separate features (X) and target (y)
# The 'Exited' column is the last column in X_train_processed (due to remainder='passthrough')
X_train_final = X_train_processed[:, :-1]
y_train_final = X_train_processed[:, -1].astype(np.long) # Target is 0/1, convert to long

X_test_final = X_test_processed[:, :-1]
y_test_final = X_test_processed[:, -1].astype(np.long)

# Create CustomDataset objects with the processed data
train_dataset = CustomDataset(X_train_final, y_train_final)
test_dataset = CustomDataset(X_test_final, y_test_final)

In [ ]:
len(train_dataset)

8000

In [ ]:
len(test_dataset)

2000

In [ ]:
train_dataset[0]

(tensor([1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 6.8600e+02,
         3.2000e+01, 6.0000e+00, 0.0000e+00, 2.0000e+00, 1.0000e+00, 1.0000e+00,
         1.7909e+05]),
 tensor(0))

In [ ]:
#create test dataset object
test_dataset=CustomDataset(X_test_final,y_test_final)

In [ ]:
#create train and test loader
train_loader= DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader= DataLoader(test_dataset,batch_size=32,shuffle=False)

In [ ]:
#define NN class
class MyNN(nn.Module):
  def __init__(self,num_features):
    super(MyNN,self).__init__()
    self.fc1=nn.Linear(num_features,64)
    self.fc2=nn.Linear(64,32)
    self.fc3=nn.Linear(32,2)
    self.relu=nn.ReLU()

  def forward(self,x):
    out=self.fc1(x)
    out=self.relu(out)
    out=self.fc2(out)
    out=self.relu(out)
    out=self.fc3(out)
    return out


In [ ]:
#set learning rate and epochs
epochs=100
learning_rate=0.1

In [ ]:
#instatiate the model
model=MyNN(X_train.shape[1])
#loss function
criterion=nn.CrossEntropyLoss()
#optimizer
optimizer=optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
#training loop

for epoch in range(epochs):

  total_epoch_loss=0
  for batch_X,batch_y in train_loader:

    #forward pass
    outputs=model(batch_X)

    #calculate loss
    loss=criterion(outputs,batch_y)

    #back pass
    optimizer.zero_grad()
    loss.backward()

    #update gradients
    optimizer.step()

    total_epoch_loss+=loss.item()

  avg_loss=total_epoch_loss/len(train_loader)
  print(f'Epoch:{epoch+1},Loss:{avg_loss}')

Epoch:1,Loss:4.92441826922883e+26
Epoch:2,Loss:0.5080739493370057
Epoch:3,Loss:0.5080539458990097
Epoch:4,Loss:0.5081568441390991
Epoch:5,Loss:0.5080132076740265
Epoch:6,Loss:0.5079415860176086
Epoch:7,Loss:0.5079465737342834
Epoch:8,Loss:0.5080040730834007
Epoch:9,Loss:0.5080682555437088
Epoch:10,Loss:0.5080682827234269
Epoch:11,Loss:0.5080001977682114
Epoch:12,Loss:0.5081576398611068
Epoch:13,Loss:0.5080509607791901
Epoch:14,Loss:0.5079020788669586
Epoch:15,Loss:0.5080599617958069
Epoch:16,Loss:0.5079063553810119
Epoch:17,Loss:0.5081354959011077
Epoch:18,Loss:0.5081114947199822
Epoch:19,Loss:0.5080006392002105
Epoch:20,Loss:0.5080978786349296
Epoch:21,Loss:0.5080628250837326
Epoch:22,Loss:0.5080958355665207
Epoch:23,Loss:0.5079253996610642
Epoch:24,Loss:0.5080974695682525
Epoch:25,Loss:0.5080270737409591
Epoch:26,Loss:0.508141725897789
Epoch:27,Loss:0.507995768070221
Epoch:28,Loss:0.5080145062208176
Epoch:29,Loss:0.5082243000268936
Epoch:30,Loss:0.5080034425258636
Epoch:31,Loss:0.507

In [ ]:
len(train_loader)

250

In [ ]:
#set model to eval mode
model.eval()

MyNN(
  (fc1): Linear(in_features=13, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
  (relu): ReLU()
)

In [ ]:
#evaluation code
total=0
correct=0

with torch.no_grad():
  for batch_X,batch_y in test_loader:
    outputs=model(batch_X)
    _, predicted=torch.max(outputs,1)
    total=total+batch_y.size(0)
    correct=correct+(predicted==batch_y).sum().item()

accuracy=100*correct/total
print(f'Accuracy:{accuracy}')


Accuracy:80.35
